In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/readme.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/collapsed_building_image0489_0.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/flood_image0097_0.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/fire_image0367_0.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/traffic_incident_image0288_1.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/traffic_incident_image0111_1.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/fire_image0227_2.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with Pose information/labels/collapsed_building_image0436_0.txt
/kaggle/input/c2a-dataset/C2A_Dataset/new_dataset3/All labels with P

In [2]:
!pip install ultralytics --quiet
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt
import glob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralyt